In [11]:
# Importing modules to access and visualise data
import xarray as xr # used for netcdf and h5 files, climate data
import h5py as h5
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfea
from pyproj import Transformer
import pyproj
import datetime
from scipy.interpolate import griddata
from scipy.stats import binned_statistic_2d
import os, sys
from matplotlib.colors import LinearSegmentedColormap
from scipy.spatial import KDTree


In [2]:
def WGS84toEASE2N(lon, lat):
    proj_EASE2N = pyproj.Proj("+proj=laea +lon_0=0 +lat_0=90 +x_0=0 +y_0=0 +ellps=WGS84 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs")
    proj_WGS84 = pyproj.Proj("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs ")
    x, y = pyproj.transform(proj_WGS84, proj_EASE2N, lon, lat)

    return x, y


def LOAD_MISR_H5(MISR_path):
    file_path = MISR_path
    file = h5.File(file_path, 'r')
    roughness = file['Roughness']['Roughness_2D_svm']
    data = np.array(roughness)
    file.close()
    
    return data


def MISR_COORDS(MISR_path):
    file_path = MISR_path
    file = h5.File(file_path, 'r')
    lon_MISR = np.array(file['GeoLocation']['Longitude'])
    lat_MISR = np.array(file['GeoLocation']['Latitude'])
    x_MISR = np.array(file['GeoLocation']['x'])
    y_MISR = np.array(file['GeoLocation']['y'])
    file.close()
    
    return lon_MISR, lat_MISR, x_MISR, y_MISR



def LOAD_MPF(MPF_path): 
    # Loads data from Sangyun Lee dataset on CPOM servers, just for month of JULY
    file_path = MPF_path
    ds = xr.open_dataset(file_path)
    data = np.array(ds['july_monthly'])
    
    return data

    

def MPF_UCL_COORDS(MPF_UCL_cords_path): 
    # reads coordinate data from sangyun lee mpf data set
    file_path = MPF_UCL_cords_path
    ds = xr.open_dataset(file_path)
    lon_MPF = ds['mp_lon']
    lat_MPF = ds['mp_lat']
        
    return np.asarray(lon_MPF), np.asarray(lat_MPF)


def interpolate_to_MISR(X_grid_in,Y_grid_in,Z_in,X_grid_out,Y_grid_out): 
    # function interpolates z data in which has shape X_in, Y_in to the same shape as X_out, Y_out
    z = griddata((X_grid_in.ravel(),Y_grid_in.ravel()),Z_in.ravel(),(X_grid_out.ravel(),Y_grid_out.ravel()),'nearest').reshape(1000,1000) 
    # to actually interpolate to MISR, need MISR x, y grid in and to reshape to 8000x8000 but can work with other shapes
    return z


# custom function to pass to statistic 
def nanmean(x):
    return np.nanmean(x)


def calculate_binned_averages(mpf, roughness, lat):

    mpf = mpf.ravel()
    roughness = roughness.ravel()
    lat = lat.ravel()

    # Define the number of bins for latitude and roughness
    num_lat_bins = 30
    num_rough_bins = 20

    # Calculate the 2D binning and get the average values of mpf in each bin
    averages, _, _, _ = binned_statistic_2d(
        lat,
        roughness,
        mpf,
        statistic = nanmean,
        bins=[num_lat_bins, num_rough_bins],
        range=[[60, 90], [0, 0.5]]
    )

    return averages


def plot_binned_stats(averages, title):

    y_tick_positions = np.linspace(0, 30, 7) # Takes default tick markings and changes them to the range of the data
    x_tick_positions = np.linspace(0, 20, 5)
    y_tick_labels = np.linspace(60, 90, 7) # Genate latitude values for 60 to 90
    x_tick_labels = np.linspace(0, 0.5, 5)

    plt.xticks(x_tick_positions, x_tick_labels) # Relabel ticks
    plt.yticks(y_tick_positions, y_tick_labels)

    plt.ylabel("Latitude")
    plt.xlabel("Roughness")

    plt.set_cmap('cubehelix_r')
    plt.pcolor(averages)
    plt.title(title)
    plt.colorbar()

    # Show the plot
    plt.show()


def plot_binned_accuracy(accuracy, colour_map, title):

    y_tick_positions = np.linspace(0, 30, 7) # Takes default tick markings and changes them to the range of the data
    x_tick_positions = np.linspace(0, 20, 5)
    y_tick_labels = np.linspace(60, 90, 7) # Genate latitude values for 60 to 90
    x_tick_labels = np.linspace(0, 0.5, 5)


    plt.xticks(x_tick_positions, x_tick_labels) # Relabel ticks
    plt.yticks(y_tick_positions, y_tick_labels)

    plt.ylabel("Latitude")
    plt.xlabel("Roughness")

    plt.pcolor(accuracy, cmap = colour_map)
    plt.title(title)
    plt.colorbar()

    # Show the plot
    plt.show()


def MASK_MPF_MISR(mpf_MISRGRID, MISR):
    # Create masks for valid data in each array
    mask1 = ~np.isnan(mpf_MISRGRID)  # Invert the NaN values to get a mask of valid data
    mask2 = ~np.isnan(MISR)

    # Create a joint mask where both arrays have valid data
    joint_mask = mask1 & mask2

    # Use the joint mask to apply the mask to both arrays and corresponding latitude
    masked_MPF_MISRGRID = np.ma.masked_array(mpf_MISRGRID, mask=~joint_mask)
    masked_MISR = np.ma.masked_array(MISR, mask=~joint_mask)
    masked_lat_MISR = np.ma.masked_array(lat_MISR, mask=~joint_mask)

    return masked_MPF_MISRGRID, masked_MISR, masked_lat_MISR


def format_date(year, month, day):
    return f"{year}-{month}-{day} 12:00:00"


def perc_diff(predicted, observed):
    return (predicted - observed) / ((predicted + observed) / 2) * 100


### Load roughness data and coords, and lat/lon paths

In [26]:
YEAR = 2017

# Retrieving coordinate data for MISR, using 2020 but all lat lon data identical. Ready transformed x and y
misr_path = f'/home/ssureen/MISR_data_monthly/April {YEAR} Roughness.h5'
lon_MISR, lat_MISR, x_MISR, y_MISR = MISR_COORDS(misr_path)

MISR = LOAD_MISR_H5(misr_path)[::8,::8]
lon_MISR = lon_MISR[::8,::8]
lat_MISR = lat_MISR[::8,::8]
x_MISR = x_MISR[::8,::8]
y_MISR = y_MISR[::8,::8]
num_points = len(MISR.ravel())
print(f'Number of points: {num_points}')
print(f'MISR shape: {np.shape(MISR)}')

lat_advected = pd.read_pickle(f'/home/htweedie/melt_ponds/data/forwarded_mpfs/lat_from_{YEAR}0401_183 days_spacing_8.pkl')
lon_advected = pd.read_pickle(f'/home/htweedie/melt_ponds/data/forwarded_mpfs/lon_from_{YEAR}0401_183 days_spacing_8.pkl')

print(f'Lat shape: {lat_advected.shape}')
print(f'Lon shape: {lon_advected.shape}')

# subset for now, then do for full dataset
date_from = format_date(YEAR, '04', '01')
date_to = format_date(YEAR, '04', '31')

subset_lat = lat_advected.loc[date_from:date_to]
subset_lon = lon_advected.loc[date_from:date_to]
print(f'Subsetted shape: {subset_lat.shape}')


Number of points: 1000000
MISR shape: (1000, 1000)
Lat shape: (184, 1000000)
Lon shape: (184, 1000000)
Subsetted shape: (30, 1000000)


In [10]:
subset_lat

,0,1,2,3,4,5,6,7,8,9,...,999990,999991,999992,999993,999994,999995,999996,999997,999998,999999
2017-04-01 12:00:00,37.418137,37.474941,37.531673,37.588333,37.644924,37.701443,37.757889,37.814266,37.870567,37.926796,...,38.026016,37.969814,37.913536,37.857185,37.800766,37.74427,37.687706,37.631069,37.574364,37.517582
2017-04-02 12:00:00,37.418137,37.474941,37.531673,37.588333,37.644924,37.701443,37.757889,37.814266,37.870567,37.926796,...,38.026016,37.969814,37.913536,37.857185,37.800766,37.74427,37.687706,37.631069,37.574364,37.517582
2017-04-03 12:00:00,37.418137,37.474941,37.531673,37.588333,37.644924,37.701443,37.757889,37.814266,37.870567,37.926796,...,38.026016,37.969814,37.913536,37.857185,37.800766,37.74427,37.687706,37.631069,37.574364,37.517582
2017-04-04 12:00:00,37.418137,37.474941,37.531673,37.588333,37.644924,37.701443,37.757889,37.814266,37.870567,37.926796,...,38.026016,37.969814,37.913536,37.857185,37.800766,37.74427,37.687706,37.631069,37.574364,37.517582
2017-04-05 12:00:00,37.418137,37.474941,37.531673,37.588333,37.644924,37.701443,37.757889,37.814266,37.870567,37.926796,...,38.026016,37.969814,37.913536,37.857185,37.800766,37.74427,37.687706,37.631069,37.574364,37.517582
2017-04-06 12:00:00,37.418137,37.474941,37.531673,37.588333,37.644924,37.701443,37.757889,37.814266,37.870567,37.926796,...,38.026016,37.969814,37.913536,37.857185,37.800766,37.74427,37.687706,37.631069,37.574364,37.517582
2017-04-07 12:00:00,37.418137,37.474941,37.531673,37.588333,37.644924,37.701443,37.757889,37.814266,37.870567,37.926796,...,38.026016,37.969814,37.913536,37.857185,37.800766,37.74427,37.687706,37.631069,37.574364,37.517582
2017-04-08 12:00:00,37.418137,37.474941,37.531673,37.588333,37.644924,37.701443,37.757889,37.814266,37.870567,37.926796,...,38.026016,37.969814,37.913536,37.857185,37.800766,37.74427,37.687706,37.631069,37.574364,37.517582
2017-04-09 12:00:00,37.418137,37.474941,37.531673,37.588333,37.644924,37.701443,37.757889,37.814266,37.870567,37.926796,...,38.026016,37.969814,37.913536,37.857185,37.800766,37.74427,37.687706,37.631069,37.574364,37.517582
2017-04-10 12:00:00,37.418137,37.474941,37.531673,37.588333,37.644924,37.701443,37.757889,37.814266,37.870567,37.926796,...,38.026016,37.969814,37.913536,37.857185,37.800766,37.74427,37.687706,37.631069,37.574364,37.517582


In [16]:
days = len(subset_lat[:][0])
start_date = [f'{YEAR}-04-01 12:00:00']
sir = np.zeros((1, num_points)) * np.nan
sir_df = pd.DataFrame(data=sir, index=start_date)

tree = KDTree(list(zip(lon_MISR.ravel(), lat_MISR.ravel())))

for day in range(days):
    lat = np.asarray(subset_lat.iloc[day])
    lon = np.asarray(subset_lon.iloc[day])

    # for each current lat and lon, find the closest original lat and lon
    _, index = tree.query(list(zip(lon, lat)), k=1)

    # create a new SIR array, with each value corresponding with a current lat/lon
    new_sir = np.zeros(len(index))
    for i in range(len(new_sir)):
        new_sir[i] = MISR[index[i]]

    


In [25]:
index.shape

(1000000,)

### Make new dataframes with x and y coords

In [28]:
days = len(subset_lat[:][0])
start_date = [f'{YEAR}-04-01 12:00:00']
x = np.zeros((days, num_points)) * np.nan
y = np.zeros((days, num_points)) * np.nan

# loop through rows
for i in range(days):
    # loop through columns
    for j in range(num_points):
        x[i,j], y[i,j] = WGS84toEASE2N(subset_lat[i][j], subset_lon[i][j])


/tmp/ipykernel_402993/599562410.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x[i,j], y[i,j] = WGS84toEASE2N(subset_lat[i][j], subset_lon[i][j])
/tmp/ipykernel_402993/1280991242.py:4: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x, y = pyproj.transform(proj_WGS84, proj_EASE2N, lon, lat)
/tmp/ipykernel_402993/599562410.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x[i,j], y[i,j] = WGS84toEASE2N(subset_lat[i][j], subset_lon[i][j])
/tmp/ipykernel_402993/1280991242.py:4: FutureWarning: This function is deprecated.

IndexError: index 30 is out of bounds for axis 0 with size 30